# Manipulating, analyzing and exporting data with tidyverse
## - Data manipulation using `dplyr` and `tidyr`
- `dplyr` is a package for helping with tabular data manipulation.
- The `tidyverse` is an "umbrella-package" that installs `tidyr`, `dplyr` and several other useful packages for data analysis, such as `ggplot2`, `tibble`, etc.
    - `tidyverse` package tries to address 3 common issues that arise when doing data analysis in R:
        - The resul from a base R function sometimes depend on the type of data.
        - R expressions are used in a non standard way, which can be confusing for new learners.
        - The existence of hidden arguments having default operations that new learners are not aware of.

In [1]:
install.packages("tidyverse")

Installing package into 'C:/Users/stef/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



package 'tidyverse' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\stef\AppData\Local\Temp\RtmpQ7dSjN\downloaded_packages


In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


- The package `dplyr` provides helper tools for the most common data manipulation tasks. It is built to work directly with data frames, many common tasks optimized by being written in a compiled language (C++). An additional feature is the ability to work directly with data stored in an external database, where only the results of the query are returned.
    - This addresses a common problem with R in that all operations are conducted in-memory and thus the amount of data you can work with is limited by available memory.
- The package `tidyr` addresses the common problem of wanting to reshape your data for plotting and usage by different R functions. 

In [2]:
surveys <- read_csv("../data_raw/portal_data_joined.csv")

Rows: 34786 Columns: 13
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (6): species_id, sex, genus, species, taxa, plot_type
dbl (7): record_id, month, day, year, plot_id, hindfoot_length, weight

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
str(surveys)

spec_tbl_df [34,786 × 13] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ record_id      : num [1:34786] 1 72 224 266 349 363 435 506 588 661 ...
 $ month          : num [1:34786] 7 8 9 10 11 11 12 1 2 3 ...
 $ day            : num [1:34786] 16 19 13 16 12 12 10 8 18 11 ...
 $ year           : num [1:34786] 1977 1977 1977 1977 1977 ...
 $ plot_id        : num [1:34786] 2 2 2 2 2 2 2 2 2 2 ...
 $ species_id     : chr [1:34786] "NL" "NL" "NL" "NL" ...
 $ sex            : chr [1:34786] "M" "M" NA NA ...
 $ hindfoot_length: num [1:34786] 32 31 NA NA NA NA NA NA NA NA ...
 $ weight         : num [1:34786] NA NA NA NA NA NA NA NA 218 NA ...
 $ genus          : chr [1:34786] "Neotoma" "Neotoma" "Neotoma" "Neotoma" ...
 $ species        : chr [1:34786] "albigula" "albigula" "albigula" "albigula" ...
 $ taxa           : chr [1:34786] "Rodent" "Rodent" "Rodent" "Rodent" ...
 $ plot_type      : chr [1:34786] "Control" "Control" "Control" "Control" ...
 - attr(*, "spec")=
  .. cols(
  ..   record_id = 

### - Selecting columns and filtering rows
####  - Selecting columns:

In [7]:
# select(data_frame, columns...)
head(select(surveys, plot_id, species_id, weight))

plot_id,species_id,weight
<dbl>,<chr>,<dbl>
2,NL,NA
2,NL,NA
2,NL,NA
2,NL,NA
2,NL,NA
2,NL,NA


In [9]:
# Select all columns EXCEPT certain ones (in this care record_id and species_id) 
head(select(surveys, -record_id, -species_id))

month,day,year,plot_id,sex,hindfoot_length,weight,genus,species,taxa,plot_type
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
7,16,1977,2,M,32,NA,Neotoma,albigula,Rodent,Control
8,19,1977,2,M,31,NA,Neotoma,albigula,Rodent,Control
9,13,1977,2,NA,NA,NA,Neotoma,albigula,Rodent,Control
10,16,1977,2,NA,NA,NA,Neotoma,albigula,Rodent,Control
11,12,1977,2,NA,NA,NA,Neotoma,albigula,Rodent,Control
11,12,1977,2,NA,NA,NA,Neotoma,albigula,Rodent,Control


#### - Filtering rows:

In [11]:
# To choose rows based on a specific criterion: filter(data_frame, criterions...)
head(filter(surveys, sex=="M", genus=="Neotoma"))

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa,plot_type
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,7,16,1977,2,NL,M,32,NA,Neotoma,albigula,Rodent,Control
72,8,19,1977,2,NL,M,31,NA,Neotoma,albigula,Rodent,Control
588,2,18,1978,2,NL,M,NA,218,Neotoma,albigula,Rodent,Control
845,5,6,1978,2,NL,M,32,204,Neotoma,albigula,Rodent,Control
990,6,9,1978,2,NL,M,NA,200,Neotoma,albigula,Rodent,Control
1164,8,5,1978,2,NL,M,34,199,Neotoma,albigula,Rodent,Control


### Pipes
1. With intermediate steps, you create a temporary data frame and use that as input to the next function, like this:

In [14]:
## This is readable, but can clutter up your workspace with lots of 
## objects that you have to name individually.

surveys2 <- filter(surveys, weight < 5)
surveys_sml <- select(surveys2, species_id, sex, weight)
head(surveys_sml)

species_id,sex,weight
<chr>,<chr>,<dbl>
PF,F,4
PF,F,4
PF,M,4
RM,F,4
RM,M,4
PF,NA,4


2. Nest functions:

In [16]:
## We get the same result as with using intermediate steps, BUT can be 
## difficult to read if too many functions are nested, as R evaluates the expression from the inside
## out. In this case: filtering then selecting.

surveys_sml <- select(filter(surveys, weight < 5), species_id, sex, weight)
head(surveys_sml)

species_id,sex,weight
<chr>,<chr>,<dbl>
PF,F,4
PF,F,4
PF,M,4
RM,F,4
RM,M,4
PF,NA,4


3. Pipes
    - Pipes are a recent additions to R. Pipes let you take the output of one function and send it directly to the next, which is useful when you need to do many things to the same dataset.
    - Pipes in R look like `%>%` and are made available via the `magrittr` package, installed automatically with `dplyr`.
    - `%>%` takes the object on its left and passes it as the first argument to the function on its right, we don't need to explicitly include the data frame as an argument when using pipes.
    - Some may find it helpful to read the pipe like the word "then".

In [19]:
## We get the same result as before

head( surveys %>%
        filter(weight < 5) %>%
        select(species_id, sex, weight) 
    )

species_id,sex,weight
<chr>,<chr>,<dbl>
PF,F,4
PF,F,4
PF,M,4
RM,F,4
RM,M,4
PF,NA,4


#### Challenge - pipes:
- Using pipes, subset the `surveys` data to include animals collected before 1995 and retain only the columns `year`, `sex`, and `weight`.

In [21]:
surveys_c1 <- surveys %>%
                filter(year < 1995) %>%
                select(year, sex, weight)

In [22]:
head(surveys_c1)

year,sex,weight
<dbl>,<chr>,<dbl>
1977,M,NA
1977,M,NA
1977,NA,NA
1977,NA,NA
1977,NA,NA
1977,NA,NA


### Mutate
- Frequently you'll want to create new columns based on the values in existing columns, for example to do unit conversions, or to find the ration of values in two columns. For this we'll user `mutate()`.
- To create a new column of weight in kg:

In [24]:
head(surveys %>%
        mutate(weight_kg = weight / 1000)
    )

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa,plot_type,weight_kg
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,7,16,1977,2,NL,M,32,NA,Neotoma,albigula,Rodent,Control,NA
72,8,19,1977,2,NL,M,31,NA,Neotoma,albigula,Rodent,Control,NA
224,9,13,1977,2,NL,NA,NA,NA,Neotoma,albigula,Rodent,Control,NA
266,10,16,1977,2,NL,NA,NA,NA,Neotoma,albigula,Rodent,Control,NA
349,11,12,1977,2,NL,NA,NA,NA,Neotoma,albigula,Rodent,Control,NA
363,11,12,1977,2,NL,NA,NA,NA,Neotoma,albigula,Rodent,Control,NA


- You can also create a second new column based on the first new column within the same call of `mutate()`:

In [28]:
surveys %>% 
        mutate(
            weight_kg = weight / 1000,
            weight_lb = weight_kg * 2.2
        ) %>%
        head()

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa,plot_type,weight_kg,weight_lb
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,7,16,1977,2,NL,M,32,NA,Neotoma,albigula,Rodent,Control,NA,NA
72,8,19,1977,2,NL,M,31,NA,Neotoma,albigula,Rodent,Control,NA,NA
224,9,13,1977,2,NL,NA,NA,NA,Neotoma,albigula,Rodent,Control,NA,NA
266,10,16,1977,2,NL,NA,NA,NA,Neotoma,albigula,Rodent,Control,NA,NA
349,11,12,1977,2,NL,NA,NA,NA,Neotoma,albigula,Rodent,Control,NA,NA
363,11,12,1977,2,NL,NA,NA,NA,Neotoma,albigula,Rodent,Control,NA,NA


In [30]:
## The first few rows of the output are full of NAs, so if we wanted to remove 
## those we could insert a filter() in the chain:

surveys %>%
    # negating the result returned from is.na(weight)
    filter(!is.na(weight)) %>% 
    mutate(weight_kg = weight / 1000) %>%
    head()

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa,plot_type,weight_kg
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
588,2,18,1978,2,NL,M,NA,218,Neotoma,albigula,Rodent,Control,0.218
845,5,6,1978,2,NL,M,32,204,Neotoma,albigula,Rodent,Control,0.204
990,6,9,1978,2,NL,M,NA,200,Neotoma,albigula,Rodent,Control,0.200
1164,8,5,1978,2,NL,M,34,199,Neotoma,albigula,Rodent,Control,0.199
1261,9,4,1978,2,NL,M,32,197,Neotoma,albigula,Rodent,Control,0.197
1453,11,5,1978,2,NL,M,NA,218,Neotoma,albigula,Rodent,Control,0.218


#### Challenge: 
- Create a new data frame from the surveys data that meets the following criteria: contains only the species_id column and a new column called hindfoot_cm containing the hindfoot_length values (currently in mm) converted to centimeters. In this hindfoot_cm column, there are no NAs and all values are less than 3.

In [22]:
surveys_cl2 <- surveys %>%
                mutate(hindfoot_cm = hindfoot_length / 10) %>%
                filter(!is.na(hindfoot_cm) & hindfoot_cm < 3) %>%
                select(species_id, hindfoot_cm)         

## - Long form (&& and ||) evaluates left to right examining only the first element of each vector and returns a logical value.
## - Short form (& and |) continues evaluation from left to right for all elements of each vector and creates (returns) 
## a vector as a result.

In [24]:
surveys_cl2 %>% head()

species_id,hindfoot_cm
<chr>,<dbl>
NL,2.8
NL,2.1
NL,2.1
NL,2.9
NL,2.9
DM,2.5


### Split-apply-combine data analysis and the `summarize()` function.
- Many data analysis tasks can be approached using the split-apply-combine paradigm:
    - Split the data into groups.
    - Apply some analysis to each group.
    - Combine the results.
- `group_by()` is often used together with `summarize()`, which collapses each group into a single-row summary of that group.
    - `group_by()` takes as arguments the column names that contain categorical variables for which you want to calculate the summary statistics.
    - Example:

In [26]:
surveys %>% 
    group_by(sex) %>% ## data is grouped by sex, but not visually separated.
    summarize(mean_weight = mean(weight, na.rm = TRUE)) ## mean_weight 
# gets calculated for each group.

sex,mean_weight
<chr>,<dbl>
F,42.17055
M,42.99538
NA,64.74257


- You can also group by multiple columns:

In [4]:
# dataset may contain NA for sex and weight

surveys %>%
    filter(!is.na(weight)) %>%
    group_by(sex, species_id) %>%

    # when mean() is called on a vector of NA the result is NaN
    
    # If we use mean() with na.rm=TRUE and by some chance in a single group 
    # all of the values are NA it means that the returned value of mean() is
    # going to be NaN. We don't want that as a result so before calculating the
    # mean value we filter (exclude) all rows in which weight is NA.

    # Example:
    # mean(c(NA,NA,NA), na.rm=TRUE) returns NaN

    summarize(mean_weight = mean(weight))

`summarise()` has grouped output by 'sex'. You can override using the `.groups` argument.


sex,species_id,mean_weight
<chr>,<chr>,<dbl>
F,BA,9.161290
F,DM,41.609685
F,DO,48.531250
F,DS,117.749548
F,NL,154.282209
F,OL,31.065817
F,OT,24.830904
F,OX,21.000000
F,PB,30.210884


- The output from the previous command doesnt run of the screen.
    - If you want to display more data, you can use the `print()` function at the end of your chain with the argument `n` specifying the number of rows to display.
    ```R
        surveys %>%
            filter(!is.na(weight)) %>%
            group_by(sex, species_id) %>%
            summarize(mean_weight = mean(weight)) %>%
            print(n = 15)            
    ```

- Once the data are grouped, you can also summarize multiple variables at the same time (and not necessarily on the same variable).
- It is sometimes useful to rearrange the result of a query to inspect the values.
    - For instance, we can sort `min_weight` to put the lighter species first (ascending order):

In [8]:
surveys %>%
    filter(!is.na(weight)) %>%
    group_by(sex, species_id) %>%
    summarize(mean_weight = mean(weight),
              min_weight = min(weight)) %>%
    arrange(min_weight) %>%
    head()

`summarise()` has grouped output by 'sex'. You can override using the `.groups` argument.


sex,species_id,mean_weight,min_weight
<chr>,<chr>,<dbl>,<dbl>
F,PF,7.974394,4
F,RM,11.102196,4
M,PF,7.885287,4
M,PP,17.184486,4
M,RM,10.123226,4
NA,PF,6.000000,4


    - to sort in descending order:

In [10]:
surveys %>%
    filter(!is.na(weight)) %>%
    group_by(sex, species_id) %>%
    summarize(mean_weight = mean(weight),
              min_weight = min(weight)) %>%
    arrange(desc(min_weight)) %>%
    head()

`summarise()` has grouped output by 'sex'. You can override using the `.groups` argument.


sex,species_id,mean_weight,min_weight
<chr>,<chr>,<dbl>,<dbl>
M,SS,130.0000,130
NA,SH,130.0000,130
NA,NL,167.6875,83
NA,DS,120.0000,78
F,SS,57.0000,57
F,SF,69.0000,46


### Counting
- If we want to count the number of rows of data for each sex we would do:

In [11]:
surveys %>%
    count(sex)

sex,n
<chr>,<int>
F,15690
M,17348
NA,1748


- `count()` is a function from the `dplyr` package.
    - `count()` is short for grouping by a variable, and summarizing it by counting the number of observations in that group.
    - Equivalent to:

In [13]:
surveys %>%
    group_by(sex) %>%
    # n() returns the number of observations in the current group
    summarize(count = n())

sex,count
<chr>,<int>
F,15690
M,17348
NA,1748


- For convenience, count() provides the sort argument:

In [14]:
surveys %>%
    count(sex, sort = TRUE)

sex,n
<chr>,<int>
M,17348
F,15690
NA,1748


In [16]:
# For comparison, this is the alternative:
surveys %>%
    group_by(sex) %>%
    summarize(count = n()) %>%
    arrange(desc(count))

sex,count
<chr>,<int>
M,17348
F,15690
NA,1748


- Counting a combination of factors:

In [ ]:
surveys %>%
    count(sex, species)
# returns all the possible variations of sex, species

- We might want to arrange the table above in:
    1. Alphabetical order of the levels of the species.
    2. Descending order of the count.

In [19]:
surveys %>%
    count(sex, species) %>%
    arrange(species, desc(n))

sex,species,n
<chr>,<chr>,<int>
F,albigula,675
M,albigula,502
NA,albigula,75
NA,audubonii,75
F,baileyi,1646
M,baileyi,1216
NA,baileyi,29
NA,bilineata,303
NA,brunneicapillus,50


#### Challenge:
    1. How many animals were caught in each plot_type surveyed?

In [21]:
surveys %>%
    count(plot_type)

plot_type,n
<chr>,<int>
Control,15611
Long-term Krat Exclosure,5118
Rodent Exclosure,4233
Short-term Krat Exclosure,5906
Spectab exclosure,3918


2. Use group_by() and summarize() to find the mean, min, and max hindfoot length for each species (using species_id). Also add the number of observations (hint: see ?n).

In [24]:
surveys %>%
    filter(!is.na(hindfoot_length)) %>%
    group_by(species_id) %>%
    summarize(
        mean = mean(hindfoot_length),
        min = min(hindfoot_length),
        max = max(hindfoot_length),
        count = n()
    )

species_id,mean,min,max,count
<chr>,<dbl>,<dbl>,<dbl>,<int>
AH,33.00000,31,35,2
BA,13.00000,6,16,45
DM,35.98235,16,50,9972
DO,35.60755,26,64,2887
DS,49.94887,39,58,2132
NL,32.29423,21,70,1074
OL,20.53261,12,39,920
OT,20.26741,13,50,2139
OX,19.12500,13,21,8


3. What was the heaviest animal measured in each year? Return the columns year, weight, species_id, and weight.

In [46]:
surveys %>%
    filter(!is.na(weight)) %>%
    group_by(year) %>%
    # - max(weight) returns the maximum weight in each group, so extracting the
    # row which contains that values becomes a trivial filter.
    filter(weight == max(weight)) %>%
    select(year, genus, species_id, weight) %>%
    arrange(year)

year,genus,species_id,weight
<dbl>,<chr>,<chr>,<dbl>
1977,Dipodomys,DS,149
1978,Neotoma,NL,232
1978,Neotoma,NL,232
1979,Neotoma,NL,274
1980,Neotoma,NL,243
1981,Neotoma,NL,264
1982,Neotoma,NL,252
1983,Neotoma,NL,256
1984,Neotoma,NL,259


### Reshaping with pivot_longer and pivot_wider
- Four rules defining a tidy dataset:
    1. Each variable has its own column.
    2. Each observation has its own row.
    3. Each value must have its own cell.
    4. Each type of observational unit forms a table.
- A record (sometimes called a row or **unit**) is a group of variables.
- `pivot_wider()`:
    - Takes three principal arguments:
        - the data
        - the names_from column variable whose values will become new column names.
        - the values_from column variable whose values will fill the new column variables.
    - Further arguments incluse `values_full` which, if set, fills in the missing values with value provided.
    - Example with `pivot_wider()`:
        - Let’s use pivot_wider() to transform surveys to find the mean weight of each genus in each plot over the entire survey period.

In [47]:
surveys_pw <- surveys %>%
    filter(!is.na(weight)) %>%
    group_by(plot_id, genus) %>%
    summarize(mean_weight = mean(weight))

`summarise()` has grouped output by 'plot_id'. You can override using the `.groups` argument.


In [54]:
View((surveys_pw))

plot_id,genus,mean_weight
<dbl>,<chr>,<dbl>
1,Baiomys,7.000000
1,Chaetodipus,22.199387
1,Dipodomys,60.232143
1,Neotoma,156.222222
1,Onychomys,27.675497
1,Perognathus,9.625000
1,Peromyscus,22.222222
1,Reithrodontomys,11.375000
2,Baiomys,6.000000


In [58]:
surveys_wide <- surveys_pw %>%

    # names_from - Column with new variable names.
    # values_from - Column of values for new variables.

    pivot_wider(names_from = genus, values_from = mean_weight, values_fill = 0)

In [63]:
View(surveys_wide %>% 
     select(plot_id, Baiomys, Chaetodipus, Dipodomys)
    )

plot_id,Baiomys,Chaetodipus,Dipodomys
<dbl>,<dbl>,<dbl>,<dbl>
1,7.000000,22.19939,60.23214
2,6.000000,25.11014,55.68259
3,8.611111,24.63636,52.04688
4,0.000000,23.02381,57.52454
5,7.750000,17.98276,51.11356
6,0.000000,24.86009,58.60531
7,0.000000,19.91603,57.36364
8,0.000000,20.47284,59.41090
9,0.000000,18.91597,57.51792


- `pivot_longer()`
    - Takes four principal arguments:
        - the data
        - the names_to column variable we wish to create from column names.
        - the values_to column variable we wish to create and fill with values.
        - cols are the name of the columns we use to make this pivot (or to drop)

In [65]:
surveys_long <- surveys_wide %>%
    # cols = -plot_id (all columns except plot_id)
    pivot_longer(names_to = "genus", values_to = "mean_weight", cols = -plot_id)  

In [62]:
View(head(surveys_long))

plot_id,genus,mean_weight
<dbl>,<chr>,<dbl>
1,Baiomys,7.00000
1,Chaetodipus,22.19939
1,Dipodomys,60.23214
1,Neotoma,156.22222
1,Onychomys,27.67550
1,Perognathus,9.62500


#### Challenge
1. Reshape the surveys data frame with year as columns, plot_id as rows, and the number of genera per plot as the values. You will need to summarize before reshaping, and use the function n_distinct() to get the number of unique genera within a particular chunk of data. It’s a powerful function! See ?n_distinct for more.

?n_distinct

In [79]:
ch_pivot_1_summarized <- surveys %>%
    group_by(plot_id, year) %>%
    summarize(n_genera = n_distinct(genus))

`summarise()` has grouped output by 'plot_id'. You can override using the `.groups` argument.


In [82]:
str(ch_pivot_1_summarized)

grouped_df [619 × 3] (S3: grouped_df/tbl_df/tbl/data.frame)
 $ plot_id : num [1:619] 1 1 1 1 1 1 1 1 1 1 ...
 $ year    : num [1:619] 1977 1978 1979 1980 1981 ...
 $ n_genera: int [1:619] 2 3 4 7 5 6 7 6 4 3 ...
 - attr(*, "groups")= tibble [24 × 2] (S3: tbl_df/tbl/data.frame)
  ..$ plot_id: num [1:24] 1 2 3 4 5 6 7 8 9 10 ...
  ..$ .rows  : list<int> [1:24] 
  .. ..$ : int [1:26] 1 2 3 4 5 6 7 8 9 10 ...
  .. ..$ : int [1:26] 27 28 29 30 31 32 33 34 35 36 ...
  .. ..$ : int [1:26] 53 54 55 56 57 58 59 60 61 62 ...
  .. ..$ : int [1:26] 79 80 81 82 83 84 85 86 87 88 ...
  .. ..$ : int [1:26] 105 106 107 108 109 110 111 112 113 114 ...
  .. ..$ : int [1:26] 131 132 133 134 135 136 137 138 139 140 ...
  .. ..$ : int [1:26] 157 158 159 160 161 162 163 164 165 166 ...
  .. ..$ : int [1:26] 183 184 185 186 187 188 189 190 191 192 ...
  .. ..$ : int [1:26] 209 210 211 212 213 214 215 216 217 218 ...
  .. ..$ : int [1:24] 235 236 237 238 239 240 241 242 243 244 ...
  .. ..$ : int [1:26] 259 2

In [80]:
reshaped_pivot_1 <- ch_pivot_1_summarized %>%
    pivot_wider(names_from = year, values_from = n_genera)

In [81]:
View(reshaped_pivot_1)

plot_id,1977,1978,1979,1980,1981,1982,1983,1984,1985,⋯,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002
<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,2,3,4,7,5,6,7,6,4,⋯,5,6,5,5,7,5,5,5,4,6
2,6,6,6,8,5,9,9,9,6,⋯,8,5,6,7,7,5,6,7,7,7
3,5,6,4,6,6,8,10,11,7,⋯,6,5,6,7,8,7,5,4,5,5
4,4,4,3,4,5,4,6,3,4,⋯,3,3,4,6,8,4,4,5,5,5
5,4,3,2,5,4,6,7,7,3,⋯,4,5,8,7,6,6,3,3,2,4
6,3,4,3,4,5,9,9,7,5,⋯,5,4,7,6,10,5,5,4,5,9
7,3,1,3,1,1,4,2,3,4,⋯,4,4,5,4,5,4,3,4,6,5
8,2,4,3,5,6,6,4,6,4,⋯,4,6,6,6,6,4,5,3,4,3
9,3,3,3,4,5,6,7,4,5,⋯,2,4,5,7,8,7,3,7,5,7


2. Now take that data frame and pivot_longer() it, so each row is a unique plot_id by year combination.

In [89]:
reshaped_pivot_2 <- reshaped_pivot_1 %>%
    pivot_longer(names_to = "year",  values_to = "n_genera", cols = -plot_id)

In [91]:
View(head(reshaped_pivot_2))

plot_id,year,n_genera
<dbl>,<chr>,<int>
1,1977,2
1,1978,3
1,1979,4
1,1980,7
1,1981,5
1,1982,6


3. The surveys data set has two measurement columns: hindfoot_length and weight. This makes it difficult to do things like look at the relationship between mean values of each measurement per year in different plot types. Let’s walk through a common solution for this type of problem. First, use pivot_longer() to create a dataset where we have a names column called measurement and a value column that takes on the value of either hindfoot_length or weight. Hint: You’ll need to specify which columns will be part of the reshape.

In [126]:
surveys_long <- surveys %>%
  pivot_longer(
          names_to = "measurement", 
          values_to = "value", 
          cols = c(hindfoot_length, weight)
  )

In [130]:
View(surveys_long)

record_id,month,day,year,plot_id,species_id,sex,genus,species,taxa,plot_type,measurement,value
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,7,16,1977,2,NL,M,Neotoma,albigula,Rodent,Control,hindfoot_length,32
1,7,16,1977,2,NL,M,Neotoma,albigula,Rodent,Control,weight,NA
72,8,19,1977,2,NL,M,Neotoma,albigula,Rodent,Control,hindfoot_length,31
72,8,19,1977,2,NL,M,Neotoma,albigula,Rodent,Control,weight,NA
224,9,13,1977,2,NL,NA,Neotoma,albigula,Rodent,Control,hindfoot_length,NA
224,9,13,1977,2,NL,NA,Neotoma,albigula,Rodent,Control,weight,NA
266,10,16,1977,2,NL,NA,Neotoma,albigula,Rodent,Control,hindfoot_length,NA
266,10,16,1977,2,NL,NA,Neotoma,albigula,Rodent,Control,weight,NA
349,11,12,1977,2,NL,NA,Neotoma,albigula,Rodent,Control,hindfoot_length,NA


4. With this new data set, calculate the average of each measurement in each year for each different plot_type. Then pivot_wider() them into a data set with a column for hindfoot_length and weight. Hint: You only need to specify the names and values columns for pivot_wider().

In [137]:
surveys_long %>%
    group_by(plot_type, year, measurement) %>%
    summarize(average_measurement = mean(value, na.rm=TRUE)) %>%
    pivot_wider(names_from=)

`summarise()` has grouped output by 'plot_type', 'year'. You can override using the `.groups` argument.


plot_type,year,measurement,average_measurement
<chr>,<dbl>,<chr>,<dbl>
Control,1977,hindfoot_length,36.13483
Control,1977,weight,50.44094
Control,1978,hindfoot_length,38.06467
Control,1978,weight,70.75169
Control,1979,hindfoot_length,36.61921
Control,1979,weight,68.05751
Control,1980,hindfoot_length,36.46993
Control,1980,weight,66.18887
Control,1981,hindfoot_length,36.97727
